# Hamming weight extractor

- Extracts the Hamming weight of the state after the SBox in the first round.
- Appends a hamming weight column to the ASCAD file.

In [3]:
import h5py
from src.data.loaders.ascad import ASCADData, ASCAD_DATA, ASCAD_DATA_VAR, ASCADDataType
from tqdm import tqdm
import numpy as np

from src.data.preprocess.hw import full_states_hw

ascad = ASCADData.random_key()
plaintext_ix = 0
full_key_ix = 1
mask_ix = 2

pm = ascad["Profiling_traces"]["metadata"]
am = ascad["Attack_traces"]["metadata"]

def get_acc(metadata):
    return np.zeros((len(metadata), 16), dtype=np.uint8)

profile_key, profile_plain, profile_mask = get_acc(pm), get_acc(pm), get_acc(pm)
attack_key, attack_plain, attack_mask = get_acc(am), get_acc(am), get_acc(am)

for ix in tqdm(range(len(pm))):
    profile_key[ix] = pm[ix][full_key_ix]
    profile_plain[ix] = pm[ix][plaintext_ix]
    profile_mask[ix] = pm[ix][mask_ix][2:]

for ix in tqdm(range(len(am))):
    attack_key[ix] = am[ix][full_key_ix]
    attack_plain[ix] = am[ix][plaintext_ix]
    attack_mask[ix] = am[ix][mask_ix][2:]

ascad.close()

100%|██████████| 100000/100000 [00:40<00:00, 2492.92it/s]


In [4]:
def affine_func(a):
    return ((a * 31) % 257) ^ 99

def unmask(states, masks):
    res = np.zeros_like(states)
    s_ix = 0
    for state, mask in tqdm(zip(states, masks)):
        fm = affine_func(mask)

        expand_fm = np.array([fm] * 10)
        res[s_ix] = state ^ expand_fm
        s_ix += 1

    return res

In [5]:
profile_hw = full_states_hw(profile_plain, profile_key)
attack_hw = full_states_hw(attack_plain, attack_key)

100%|██████████| 100000/100000 [00:34<00:00, 2926.45it/s]


In [6]:
profile_unmask = unmask(profile_hw, profile_mask)
attack_unmask = unmask(attack_hw, attack_mask)

200000it [00:01, 100491.84it/s]
100000it [00:00, 101253.36it/s]


In [7]:
np.mean(profile_hw), np.mean(attack_hw)

(4.0002320625, 4.0002253125)

In [8]:
wrt = h5py.File(f"{ASCAD_DATA}{ASCAD_DATA_VAR}/{ASCADDataType.default}.h5", 'r+')

for grp_name, grp_data in [("Profiling_traces", profile_hw), ("Attack_traces", attack_hw)]:
    grp = wrt[grp_name]

    if "hw_unmasked" in grp:
        del grp["hw_unmasked"]
    grp["hw_unmasked"] = grp_data

wrt.close()

chk = h5py.File(f"{ASCAD_DATA}{ASCAD_DATA_VAR}/{ASCADDataType.default}.h5", 'r')
print(list(chk["Profiling_traces"]))
print(list(chk["Attack_traces"]))
chk.close()


['aes_r1b3', 'hamming_weights', 'hw_unmasked', 'labels', 'labels_hw0', 'labels_mask', 'metadata', 'traces']
['aes_r1b3', 'hamming_weights', 'hw_unmasked', 'labels', 'labels_hw0', 'labels_mask', 'metadata', 'traces']
